# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sqlalchemy import create_engine

# For machine learning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

# For nlp
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterDataBase.db')
df = pd.read_sql_table('DisasterDataBase', engine)
X = df['message'] # Message column
Y = df.iloc[:, 4:] # Classification label

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [4]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Function: split text into words and return the root form of the words, and removing 
    Args:
      text(str): the message
    Return:
      clean_tokens - tokens after cleaning, tokenization and lemmatization of the text
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detecting URLs and removing punctuation
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Normalize, tokenize and lemmatize
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
normal_pipeline = Pipeline ([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
])

normal_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
%%time 

# Spliting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# Fiting the Random Forest Classifier 
normal_pipeline.fit(X_train, Y_train)

CPU times: user 1min 4s, sys: 105 ms, total: 1min 5s
Wall time: 1min 5s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = normal_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(Y.columns)):
    print("===========================",Y_test.columns[i], '===============================')
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print(f'Accuracy {accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i]):.3f}\n\n')
    print(f'F1: {f1_score(Y_test.iloc[:, i].values, Y_pred[:, i],average="weighted"):.3f}\n\n')

accuracy = (Y_pred == Y_test.values).mean()
print(f'The model accuracy is {accuracy:.3f}')

=========================== related ===============================
             precision    recall  f1-score   support

          0       0.61      0.36      0.46      1522
          1       0.82      0.93      0.87      4993
          2       0.50      0.05      0.09        39

avg / total       0.77      0.79      0.77      6554

Accuracy 0.793


F1: 0.771


=========================== request ===============================
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5455
          1       0.83      0.37      0.51      1099

avg / total       0.88      0.88      0.86      6554

Accuracy 0.881


F1: 0.862


=========================== offer ===============================
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

Accuracy 0.996


F1: 0.993


=============

In [9]:
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.37      0.51      1099
               request       0.00      0.00      0.00        29
                 offer       0.75      0.50      0.60      2759
           aid_related       0.69      0.07      0.12       517
          medical_help       0.68      0.04      0.08       299
      medical_products       0.67      0.08      0.14       199
     search_and_rescue       0.00      0.00      0.00       125
              security       0.50      0.05      0.10       226
              military       0.00      0.00      0.00         0
           child_alone       0.89      0.20      0.32       403
                 water       0.79      0.32      0.45       721
                  food       0.78      0.18      0.29       540
               shelter       0.63      0.12      0.20       101
              clothing       0.83      0.03      0.06       151
                 money       0.00      

In [10]:
# The percentual of data distribution on features
Y.sum()/len(Y)

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
normal_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd82b9ef598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
parameters = {
    'vect__max_features': (None, 5000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50,100],
    'clf__estimator__min_samples_split': [2, 3],
}

normal_pipeline_cv = GridSearchCV(normal_pipeline, param_grid = parameters, cv=2, n_jobs=-1, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
%%time 

normal_pipeline_cv.fit(X_train, Y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=None 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=None, score=0.22886786695148, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=None 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=None, score=0.22988505747126436, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=5000 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=5000, score=0.24463432000813753, total= 1.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=5000 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_features=5000, score=0.24158274844878447, total= 1.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, vect__max_features=None 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, vect__max_features=None, score=0.2317160004068762, total= 1.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, vect__max_features=None 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=False, vect__max_features=None, score=0.222764723832773

[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, tfidf__use_idf=False, vect__max_features=5000, score=0.23863289594140982, total= 2.6min


[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed: 80.9min finished


CPU times: user 1h 27min 17s, sys: 2.18 s, total: 1h 27min 19s
Wall time: 1h 27min 32s


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (None, 5000), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [14]:
#finding the best paramesters based on grip search
print('Best Parameters:', normal_pipeline_cv.best_params_)

Best Parameters: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': True, 'vect__max_features': 5000}


In [15]:
#building tuned model
tuned_model = normal_pipeline_cv.best_estimator_
print (normal_pipeline_cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


In [17]:

Y_pred = tuned_model.predict(X_test) 
          
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred]), target_names = Y.columns))

accuracy = (Y_pred == Y_test.values).mean()
print(f'The model accuracy is {accuracy:.3f}')

                        precision    recall  f1-score   support

               related       0.86      0.48      0.62      1099
               request       0.00      0.00      0.00        29
                 offer       0.78      0.64      0.70      2759
           aid_related       0.64      0.10      0.18       517
          medical_help       0.81      0.13      0.22       299
      medical_products       0.69      0.11      0.19       199
     search_and_rescue       0.20      0.01      0.02       125
              security       0.58      0.07      0.12       226
              military       0.00      0.00      0.00         0
           child_alone       0.87      0.37      0.52       403
                 water       0.81      0.65      0.72       721
                  food       0.83      0.43      0.56       540
               shelter       0.80      0.12      0.21       101
              clothing       0.67      0.03      0.05       151
                 money       0.50      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [19]:
new_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize))
            , ('tfidf', TfidfTransformer())]))
        , ('starting_verb', StartingVerbExtractor())]))
    , ('clf', RandomForestClassifier())])

new_pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_start=False))], 'features': Featu

In [20]:
%%time

new_pipeline.fit(X_train, Y_train)

CPU times: user 58 s, sys: 767 ms, total: 58.8 s
Wall time: 59.4 s


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [21]:
Y_pred = new_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(Y.columns)):
    print(f'Category: {Y.columns[i]}')
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print(f'Accuracy {accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i]):.3f}\n\n')
    print(f'F1: {f1_score(Y_test.iloc[:, i].values, Y_pred[:, i],average="weighted"):.3f}\n\n')

accuracy = (Y_pred == Y_test.values).mean()
print(f'The model accuracy is {accuracy:.3f}')

Category: related
             precision    recall  f1-score   support

          0       0.56      0.46      0.50      1522
          1       0.84      0.89      0.87      4993
          2       0.56      0.23      0.33        39

avg / total       0.77      0.79      0.78      6554

Accuracy 0.788


F1: 0.779


Category: request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5455
          1       0.84      0.39      0.53      1099

avg / total       0.88      0.89      0.87      6554

Accuracy 0.886


F1: 0.868


Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

Accuracy 0.996


F1: 0.993


Category: aid_related
             precision    recall  f1-score   support

          0       0.68      0.91      0.78      3795
          1       0.77      0.42  

### 9. Export your model as a pickle file

In [22]:
pickle.dump(tuned_model, open('tuned_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.